In [9]:
import os
from dotenv import load_dotenv
from langchain.tools import tool
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [10]:
@tool
def multiply_tool(x: int, y: int) -> int:
    """Multiply two numbers."""
    return x * y

In [11]:
multiply_tool.args

{'x': {'title': 'X', 'type': 'integer'},
 'y': {'title': 'Y', 'type': 'integer'}}

In [12]:
LLM = ChatGroq(groq_api_key=os.environ.get("GROQ_API_KEY"), model="openai/gpt-oss-120b", temperature=1.2)
LLM_with_tool = LLM.bind_tools([multiply_tool])

In [13]:
LLM_with_tool.invoke('Hi how are you')

AIMessage(content="I'm doing great, thanks for asking! How can I help you today?", additional_kwargs={'reasoning_content': 'We need to respond. Friendly. No special request.'}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 125, 'total_tokens': 161, 'completion_time': 0.073367534, 'prompt_time': 0.027477382, 'queue_time': 0.042907028, 'total_time': 0.100844916}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_5c888d599f', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--20bd9cd8-dbc6-451b-8025-c008fea66660-0', usage_metadata={'input_tokens': 125, 'output_tokens': 36, 'total_tokens': 161})

In [14]:
query = HumanMessage('can you multiply 3 with 1000')